<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [1]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [2]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.18.5
Pandas:  1.0.5


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [3]:
# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = None)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = 0)
print(bikes.head())

FileNotFoundError: [Errno 2] File dat/bikeshare.csv does not exist: 'dat/bikeshare.csv'

?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [34]:
#ANSWER:
bikes = pd.read_table('../../../DATA/bikeshare.csv', header = 0, delimiter=',')
print(bikes.head())
#NOTE.1: the "../../DATA" refers to the local file location
#NOTE.2: delimiter refers to the seperator which differentiates the header lables. 

   instant      dteday  season  yr  mnth  hr  holiday  weekday  workingday  \
0        1  2011-01-01       1   0     1   0        0        6           0   
1        2  2011-01-01       1   0     1   1        0        6           0   
2        3  2011-01-01       1   0     1   2        0        6           0   
3        4  2011-01-01       1   0     1   3        0        6           0   
4        5  2011-01-01       1   0     1   4        0        6           0   

   weathersit  temp   atemp   hum  windspeed  casual  registered  cnt  
0           1  0.24  0.2879  0.81        0.0       3          13   16  
1           1  0.22  0.2727  0.80        0.0       8          32   40  
2           1  0.22  0.2727  0.80        0.0       5          27   32  
3           1  0.24  0.2879  0.75        0.0       3          10   13  
4           1  0.24  0.2879  0.75        0.0       0           1    1  


Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [4]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [22]:
df = pd.read_excel('../../../DATA/Iris.xls')
df

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,1,0.2,1.4,3.5,5.1,Setosa
1,1,0.2,1.4,3.0,4.9,Setosa
2,1,0.2,1.3,3.2,4.7,Setosa
3,1,0.2,1.5,3.1,4.6,Setosa
4,1,0.2,1.4,3.6,5.0,Setosa
...,...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7,Verginica
146,3,1.9,5.0,2.5,6.3,Verginica
147,3,2.0,5.2,3.0,6.5,Verginica
148,3,2.3,5.4,3.4,6.2,Verginica


So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [0]:
#ANSWER: I think the file has been updated as there is no embedded table of aggregates on the same sheet, making this question void.

### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [10]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_month.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2020-09-15T03:41:28.012Z,55.9268,158.4539,344.0,6.4,mww,NaN,29,2.858,0.73,...,2020-09-17T04:11:41.155Z,"15 km W of Esso, Russia",earthquake,7.4,1.9,0.028,120,reviewed,us,us
1,2020-09-11T07:35:57.894Z,-21.3928,-69.8943,51.0,6.3,mww,NaN,41,0.072,1.39,...,2020-09-13T08:04:13.078Z,"83 km NNE of Tocopilla, Chile",earthquake,3.0,1.8,0.052,35,reviewed,us,us
2,2020-09-09T06:00:13.113Z,40.2621,-74.2456,5.0,3.1,mb_lg,NaN,140,0.485,0.45,...,2020-09-16T14:30:49.489Z,"2 km SSE of East Freehold, New Jersey",earthquake,2.8,2.0,0.054,90,reviewed,us,us
3,2020-09-08T08:45:27.723Z,51.8170,-0.6649,10.0,3.9,mb,NaN,104,3.314,1.01,...,2020-09-14T14:34:29.723Z,"2 km SW of Pitstone, United Kingdom",earthquake,6.0,2.0,0.194,7,reviewed,us,us
4,2020-09-06T15:23:43.148Z,6.2693,125.8285,120.0,6.3,mww,NaN,29,0.832,1.22,...,2020-09-10T17:17:41.108Z,"17 km E of Talagutong, Philippines",earthquake,7.2,1.9,0.045,47,reviewed,us,us


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [11]:
url = 'http://www.fdic.gov/bank/individual/failed/banklist.html'
df = pd.read_html(url)
df

[                             Bank Name           City  ST   CERT  \
 0                 The First State Bank  Barboursville  WV  14361   
 1                   Ericson State Bank        Ericson  NE  18265   
 2     City National Bank of New Jersey         Newark  NJ  21111   
 3                        Resolute Bank         Maumee  OH  58317   
 4                Louisa Community Bank         Louisa  KY  58112   
 ..                                 ...            ...  ..    ...   
 556                 Superior Bank, FSB       Hinsdale  IL  32646   
 557                Malta National Bank          Malta  OH   6629   
 558    First Alliance Bank & Trust Co.     Manchester  NH  34264   
 559  National State Bank of Metropolis     Metropolis  IL   3815   
 560                   Bank of Honolulu       Honolulu  HI  21029   
 
                    Acquiring Institution       Closing Date  
 0                         MVB Bank, Inc.      April 3, 2020  
 1             Farmers and Merchants Bank  F

#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [12]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,integer,datetime,category
0,5,2015-01-01 00:00:00,0
1,5,2015-01-01 00:00:01,0
2,9,2015-01-01 00:00:02,0
3,6,2015-01-01 00:00:03,0
4,6,2015-01-01 00:00:04,0


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [37]:
#ANSWER
bikes.shape
#gives me (rows, columns)

(17379, 17)

In [38]:
bikes
#complete dataset visual

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,17375,2012-12-31,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642,11,108,119
17375,17376,2012-12-31,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642,8,81,89
17376,17377,2012-12-31,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642,7,83,90
17377,17378,2012-12-31,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343,13,48,61


What are the column names in `bikes`?

In [39]:
#ANSWER
bikes.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

What are the data types of these columns?

In [40]:
#ANSWER
bikes.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

What is the (row) index for this DataFrame?

In [41]:
#ANSWER
bikes.index
#not sure if this is what is being asked?

RangeIndex(start=0, stop=17379, step=1)

https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [43]:
#ANSWER
bikes['season']

0        1
1        1
2        1
3        1
4        1
        ..
17374    1
17375    1
17376    1
17377    1
17378    1
Name: season, Length: 17379, dtype: int64

In [44]:
bikes[['season']]

,season
0,1
1,1
2,1
3,1
4,1
...,...
17374,1
17375,1
17376,1
17377,1


How would we use object notation to show the first 4 rows of `atemp`?

In [57]:
#ANSWER
bikes[['atemp']][0:4]

,atemp
0,0.2879
1,0.2727
2,0.2727
3,0.2879


In [214]:
bikes.atemp[:4] #alternative format

0    0.2879
1    0.2727
2    0.2727
3    0.2879
Name: atemp, dtype: float64

Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [60]:
bikes[[0]]
bikes[0]
bikes[0,0]
bikes[[0,0]]

KeyError: "None of [Int64Index([0], dtype='int64')] are in the [columns]"

What is the correct way to access the 1st row of the DataFrame by its index?

In [87]:
#ANSWER
bikes.iloc[[0]]

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16


In [216]:
bikes[:1] #alternative answer

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,atemp_level
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16,mild


What is the correct way to access the 2nd column of the DataFrame by its index?

In [89]:
#ANSWER
bikes.iloc[:,1]

0        2011-01-01
1        2011-01-01
2        2011-01-01
3        2011-01-01
4        2011-01-01
            ...    
17374    2012-12-31
17375    2012-12-31
17376    2012-12-31
17377    2012-12-31
17378    2012-12-31
Name: dteday, Length: 17379, dtype: object

## Handling Missing Values

What is the Pandas `isnull` function for? 

?
ANSWER:

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [90]:
bikes.isnull().head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [104]:
#ANSWER
bikes.isnull().count() #per column? This is not quite what I want to achieve

instant       17379
dteday        17379
season        17379
yr            17379
mnth          17379
hr            17379
holiday       17379
weekday       17379
workingday    17379
weathersit    17379
temp          17379
atemp         17379
hum           17379
windspeed     17379
casual        17379
registered    17379
cnt           17379
dtype: int64

In [107]:
bikes.isna().count() #same, but using isna

instant       17379
dteday        17379
season        17379
yr            17379
mnth          17379
hr            17379
holiday       17379
weekday       17379
workingday    17379
weathersit    17379
temp          17379
atemp         17379
hum           17379
windspeed     17379
casual        17379
registered    17379
cnt           17379
dtype: int64

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [217]:
#ANSWER
bikes.isnull().sum()

instant        0
dteday         0
season         0
yr             0
mnth           0
hr             0
holiday        0
weekday        0
workingday     0
weathersit     0
temp           0
atemp          0
hum            0
windspeed      0
casual         0
registered     0
cnt            0
atemp_level    0
dtype: int64

What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: Marking a data point as invalid.

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [93]:
#ANSWER
def divide(num,denom):
    try:
        return num/denom
    except:
        return 0 #this means that if i get an error, instead of returning "error" it returns something else.

In [92]:
divide(7,0) #this is an example demonstrated in class

0

In [ ]:
#ANSWER from solutions
def divide(dividend, divisor):
    if divisor == 0:
        quotient = np.nan
    else:
        quotient = dividend / divisor
    return (quotient)

print(divide(1, 0))

Apply the Pandas `isna` function to the following data objects:

In [110]:
x = 2.3
y = np.nan
print(x, y)

2.3 nan


In [126]:
#ANSWER

In [123]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

[[ 1. nan  3.]
 [ 4.  5. nan]]


In [218]:
#ANSWER
pd.isna(array)

array([[False,  True, False],
       [False, False,  True]])

How is the pandas I/O parameter `na_values` used?

? ANSWER: 

In [ ]:
#solution notes: For converting specially coded strings to NaN on input (e.g. "###", "N/A", "NULL")

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [133]:
#ANSWER
bikes['season'].value_counts()

3    4496
2    4409
1    4242
4    4232
Name: season, dtype: int64

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [1]:
#ANSWER
bikes['windspeed'].range #what am i doing wrong?

NameError: name 'bikes' is not defined

In [219]:
#Actual Solution:
print('instant:', bikes['instant'].min(), 'to', bikes['instant'].max())
print('dteday:', bikes['dteday'].min(), 'to', bikes['dteday'].max())
print('windspeed:', bikes['windspeed'].min(), 'to', bikes['windspeed'].max())

instant: 1 to 17379
dteday: 2011-01-01 to 2012-12-31
windspeed: 0.0 to 0.8507


Compute and print the overall minimum and maximum of the numeric data columns:

In [222]:
bikes_min, bikes_max = (min(bikes.min(numeric_only=True)), max(bikes.max(numeric_only=True)))
bikes_min, bikes_max

(0.0, 17379.0)

### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [149]:
bikes['atemp'].quantile(0.5)

0.4848

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [156]:
#ANSWER
bikes['atemp'].quantile([0.10,0.25,0.5,0.75,0.9])

0.10    0.2424
0.25    0.3333
0.50    0.4848
0.75    0.6212
0.90    0.6970
Name: atemp, dtype: float64

### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [157]:
type(bikes['atemp'])

pandas.core.series.Series

In [158]:
bikes.sample(5)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
14934,14935,2012-09-19,3,1,9,11,0,3,1,1,0.60,0.6212,0.43,0.2537,44,203,247
17116,17117,2012-12-20,4,1,12,23,0,4,1,3,0.40,0.4091,0.82,0.2985,0,13,13
7458,7459,2011-11-12,4,0,11,9,0,6,0,1,0.30,0.3030,0.61,0.1642,18,142,160
13562,13563,2012-07-24,3,1,7,7,0,2,1,1,0.70,0.6667,0.74,0.1642,24,492,516
5696,5697,2011-08-30,3,0,8,19,0,2,1,1,0.66,0.6212,0.50,0.0896,68,397,465


Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [163]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)   

What is `atemp_level`?

In [165]:
#ANSWER: not quite sure... categories that have been established to group the temperate data in 4 bins
# split equally into 4? or 4 by quatitative sorting value?

Here is a random sample of `atemp_level`:

In [161]:
atemp_level.sample(5)          

17126    (0.25, 0.5]
13024    (0.5, 0.75]
15282    (0.5, 0.75]
15439    (0.5, 0.75]
7041     (0.25, 0.5]
Name: atemp, dtype: category
Categories (4, interval[float64]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [164]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)          

476      cool
15829    warm
5935     warm
2052     cool
7996     mild
Name: atemp, dtype: category
Categories (4, object): [cool < mild < warm < hot]

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [167]:
#ANSWER
bikes['atemp_level'] = atemp_level

In [185]:
bikes
# new column has been succesfully added

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,atemp_level
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16,mild
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40,mild
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32,mild
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13,mild
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1,mild
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,17375,2012-12-31,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642,11,108,119,mild
17375,17376,2012-12-31,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642,8,81,89,mild
17376,17377,2012-12-31,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642,7,83,90,mild
17377,17378,2012-12-31,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343,13,48,61,mild


In [226]:
bikes['season']==2,['atemp_level']=="mild"
#thought process above, adapted to include count() below:
bikes[(bikes.atemp_level == 'mild') & (bikes.season == 2)].season.count()

1829

*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [191]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

In [198]:
# Cleaning Data
a

array([0., 0., 0.])

In [199]:
b

array([[1., 1., 1.]])

In [200]:
c

array([[[9, 9, 2, 9],
        [7, 3, 4, 2],
        [3, 5, 2, 5]],

       [[2, 7, 4, 4],
        [4, 4, 1, 3],
        [9, 6, 1, 8]]])

In [201]:
d

array([0, 1, 2, 3])

In [202]:
e

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

## Load Data

Load rock.csv and clean the dataset.

In [203]:
rock = pd.read_table('../../../DATA/rock.csv', header = 0, delimiter=',')
print(rock.head())

               Song Clean ARTIST CLEAN Release Year  \
0        Caught Up in You  .38 Special         1982   
1            Fantasy Girl  .38 Special          NaN   
2         Hold On Loosely  .38 Special         1981   
3  Rockin' Into the Night  .38 Special         1980   
4       Art For Arts Sake         10cc         1975   

                                COMBINED  First?  Year?  PlayCount  F*G  
0        Caught Up in You by .38 Special       1      1         82   82  
1            Fantasy Girl by .38 Special       1      0          3    0  
2         Hold On Loosely by .38 Special       1      1         85   85  
3  Rockin' Into the Night by .38 Special       1      1         18   18  
4              Art For Arts Sake by 10cc       1      1          1    1  


## Check Column Names

Check column names and clean.

In [204]:
rock.columns

Index(['Song Clean', 'ARTIST CLEAN', 'Release Year', 'COMBINED', 'First?',
       'Year?', 'PlayCount', 'F*G'],
      dtype='object')

In [205]:
rock.rename(columns={'Song Clean':'Song','ARTIST CLEAN':'Artist','COMBINED':'Combined','PlayCount':'Play Count'}, inplace=True)

In [206]:
rock.columns

Index(['Song', 'Artist', 'Release Year', 'Combined', 'First?', 'Year?',
       'Play Count', 'F*G'],
      dtype='object')

In [ ]:
#this process would be gruelling for larger data sets, see solutions for a refined method.

# Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

In [210]:
rock['Release Year'].isnull()

0       False
1        True
2       False
3       False
4       False
        ...  
2225     True
2226    False
2227    False
2228    False
2229    False
Name: Release Year, Length: 2230, dtype: bool

In [211]:
rock['Release Year'].fillna('0', inplace = True)

In [212]:
rock['Release Year']

0       1982
1          0
2       1981
3       1980
4       1975
        ... 
2225       0
2226    1981
2227    1975
2228    1983
2229    1973
Name: Release Year, Length: 2230, dtype: object

In [229]:
#check changes have been made
rock.isnull().sum() 
#note that release year now has 0 null values

Song            0
Artist          0
Release Year    0
Combined        0
First?          0
Year?           0
Play Count      0
F*G             0
dtype: int64

# Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 

In [213]:
rock.dtypes

Song            object
Artist          object
Release Year    object
Combined        object
First?           int64
Year?            int64
Play Count       int64
F*G              int64
dtype: object

In [237]:
rock.info() #see: Release Year is an object, should be an float/integer.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2230 entries, 0 to 2229
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Song          2230 non-null   object 
 1   Artist        2230 non-null   object 
 2   Release Year  2229 non-null   float64
 3   Combined      2230 non-null   object 
 4   First?        2230 non-null   int64  
 5   Year?         2230 non-null   int64  
 6   Play Count    2230 non-null   int64  
 7   F*G           2230 non-null   int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 139.5+ KB


In [232]:
rock['Release Year'].value_counts() #SONGFACTS.COM isnt a year

0                577
1973             104
1977              83
1975              83
1970              81
1971              75
1969              72
1980              70
1978              64
1979              63
1981              61
1967              61
1983              60
1976              56
1982              54
1984              51
1972              50
1974              48
1968              46
1987              39
1985              39
1986              37
1991              34
1989              32
1966              30
1988              29
1965              28
1994              25
1990              22
1993              19
1964              14
1992              14
1999              13
1995              10
1997               9
1996               9
1963               9
1998               6
2002               6
2012               5
2004               5
2005               5
2001               4
1962               3
2007               3
2003               3
2008               3
2000         

In [233]:
# Unsure: check solutions.
# Create a mask to find all rows with SONGFACTS.COM
song_facts_mask = rock['Release Year'] == 'SONGFACTS.COM'
rock.loc[song_facts_mask, 'Release Year'] = np.NaN

In [234]:
# Convert release year object to numeric values
rock['Release Year'] = pd.to_numeric(rock['Release Year'])

In [236]:
# Manual check
rock.dtypes

Song             object
Artist           object
Release Year    float64
Combined         object
First?            int64
Year?             int64
Play Count        int64
F*G               int64
dtype: object

# Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [238]:
def check_min_max(df):  # Check min and max of columns
    print(df.describe().T[['min', 'max']])

In [239]:
check_min_max(rock) #min. value of release year is 0, which is not valid

              min     max
Release Year  0.0  2014.0
First?        1.0     1.0
Year?         0.0     1.0
Play Count    0.0   142.0
F*G           0.0   142.0


In [240]:
# Find all rows with 0 by creating another mask
song_facts_mask = rock['Release Year'] == 0
rock.loc[song_facts_mask, 'Release Year'] = np.NaN

In [241]:
check_min_max(rock) #see min is now 1071 which is still incorrect

                 min     max
Release Year  1071.0  2014.0
First?           1.0     1.0
Year?            0.0     1.0
Play Count       0.0   142.0
F*G              0.0   142.0


In [243]:
#so:
song_facts_mask = rock['Release Year'] == 1071
rock.loc[song_facts_mask, 'Release Year'] = np.NaN

In [245]:
check_min_max(rock) #seems legit!

                 min     max
Release Year  1955.0  2014.0
First?           1.0     1.0
Year?            0.0     1.0
Play Count       0.0   142.0
F*G              0.0   142.0


# Write Some Functions

In [ ]:
#ok, so this is where I ran out of time and brain power. Following answers have been established with help frmo the solution handout

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [246]:
rock.dtypes

Song             object
Artist           object
Release Year    float64
Combined         object
First?            int64
Year?             int64
Play Count        int64
F*G               int64
dtype: object

In [247]:
def check_song(row):
    print('Song: ', row['Song'])
    print('Artist: ', row['Artist'])
    print('Released before 1970: ', row['Release Year'] < 1970)
    print('\n')

In [248]:
rock.iloc[0]

Song                           Caught Up in You
Artist                              .38 Special
Release Year                               1982
Combined        Caught Up in You by .38 Special
First?                                        1
Year?                                         1
Play Count                                   82
F*G                                          82
Name: 0, dtype: object

In [249]:
#example: check any row (first)
check_song(rock.iloc[0])

Song:  Caught Up in You
Artist:  .38 Special
Released before 1970:  False




In [250]:
# Check first five rows with new function:
rock.head(5).apply(check_song, axis=1)

Song:  Caught Up in You
Artist:  .38 Special
Released before 1970:  False


Song:  Fantasy Girl
Artist:  .38 Special
Released before 1970:  False


Song:  Hold On Loosely
Artist:  .38 Special
Released before 1970:  False


Song:  Rockin' Into the Night
Artist:  .38 Special
Released before 1970:  False


Song:  Art For Arts Sake
Artist:  10cc
Released before 1970:  False




0    None
1    None
2    None
3    None
4    None
dtype: object

## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

In [251]:
def convert_to_float(column):
    column = pd.to_numeric(column, errors='coerce')
    return column

## Apply these functions to your dataset

In [254]:
rock.apply(convert_to_float) #apply float conversion function to Rock.

,Song,Artist,Release Year,Combined,First?,Year?,Play Count,F*G
0,NaN,NaN,1982.0,NaN,1,1,82,82
1,NaN,NaN,NaN,NaN,1,0,3,0
2,NaN,NaN,1981.0,NaN,1,1,85,85
3,NaN,NaN,1980.0,NaN,1,1,18,18
4,NaN,NaN,1975.0,NaN,1,1,1,1
...,...,...,...,...,...,...,...,...
2225,NaN,NaN,NaN,NaN,1,0,1,0
2226,NaN,NaN,1981.0,NaN,1,1,32,32
2227,NaN,NaN,1975.0,NaN,1,1,109,109
2228,NaN,NaN,1983.0,NaN,1,1,1,1


## Describe the new float-only DataFrame.

In [255]:
rock.describe().T
#all columns are now float

,count,mean,std,min,25%,50%,75%,max
Release Year,1651.0,1978.569352,9.309780,1955.0,1971.0,1977.0,1984.0,2014.0
First?,2230.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
Year?,2230.0,0.741256,0.438043,0.0,0.0,1.0,1.0,1.0
Play Count,2230.0,16.872646,25.302972,0.0,1.0,4.0,21.0,142.0
F*G,2230.0,15.048430,25.288366,0.0,0.0,3.0,18.0,142.0




> 
>
>




>


>


>




---



---



> > > > > > > > > © 2019 Institute of Data


---



---



